In [1]:
import numpy as np
from collections import UserList

In [2]:
class AutoCorrList(UserList):

    def __init__(self, initlist=None):
        if initlist == None:
            super().__init__()
        elif type(initlist[0]) == type(np.ndarray([])):
            super().__init__(initlist)
        else:
            raise ValueError(f"need to data type = numpy.ndarray: actual {type(initlist[0])}")

    def __getitem__(self, i):
        if isinstance(i, slice):
            raise NotImplementedError("accessor for AutoCorrList")
            # return self.__class__(self.data[i])
        else:
            if i < 0:
                return self.data[-i].T
            return self.data[i]

    def __setitem__(self, i, item):
        if i < 0:
            raise ValueError(f"need index >= 0: actual {i}")
        self.data[i] = item

In [3]:
d = 2
p = 4
mat = [np.random.randn(d, d) for i in range(0, p+1)]
autocorrs = AutoCorrList(mat)
coeffs = [np.random.randn(d, d) for i in range(1, p+1)]

In [4]:
e = np.eye(d, d)


def Phi(coeffs, k):
    ret = autocorrs[k]
    for l in range(1, p+1):
        ret = ret + coeffs[l-1] @ autocorrs[k-l]
    return ret


def grad_elem(coeffs, l, i, j):
    assert l > 0
    ret = np.zeros_like(autocorrs[0])
    for k in range(1, p+1):
        ret = ret + autocorrs[k-l] @ (Phi(coeffs, k).T)
    return e[j].T @ ret @ e[i]


def gradient(coeffs):
    return [np.array([[grad_elem(coeffs, l, i, j) for j in range(d)] for i in range(d)]) for l in range(1, p+1)]


def loss(coeffs):
    ret = 0.0
    for l in range(1,p+1):
        P = Phi(coeffs, l)
        ret += np.trace(P.T @ P)
    return ret

In [5]:
iters = 10000
lr = 0.2
it_per_epoch = iters / 20

coeffs = [np.random.randn(d, d) for i in range(1, p+1)]
# coeffs = [expect]
print(coeffs)
print(f"init loss: {loss(coeffs)}")

for i in range(iters):
    grads = gradient(coeffs)
    for l in range(1, p+1):
        coeffs[l-1] -= lr * grads[l-1]
        
    if i % it_per_epoch == 0:
        print(f"loss at iter #{i}: {loss(coeffs)}")

coeffs

[array([[ 0.89731621,  1.59543423],
       [-0.34464134,  0.91521966]]), array([[ 0.97779386,  0.74693378],
       [ 0.30230158, -0.19082967]]), array([[ 0.32635422, -1.39134243],
       [ 1.46963933,  0.38730605]]), array([[-1.28246176, -1.47510654],
       [ 0.38851965,  0.12897621]])]
init loss: 59.69359258979523
loss at iter #0: 9.141061495127683
loss at iter #500: 5242420021.482582
loss at iter #1000: 1.864193198218287e+19
loss at iter #1500: 6.6290306118977e+28
loss at iter #2000: 2.357268919094681e+38
loss at iter #2500: 8.382397189351785e+47
loss at iter #3000: 2.9807622741251704e+57
loss at iter #3500: 1.0599526047434852e+67
loss at iter #4000: 3.769168491077467e+76
loss at iter #4500: 1.3403081468505087e+86
loss at iter #5000: 4.766106722918924e+95
loss at iter #5500: 1.6948172215196343e+105
loss at iter #6000: 6.026733309489049e+114
loss at iter #6500: 2.143093303662421e+124
loss at iter #7000: 7.620793342508254e+133
loss at iter #7500: 2.709937596742489e+143
loss at iter #8

[array([[3.44118263e+94, 5.05793995e+93],
        [3.70416403e+94, 5.44447687e+93]]),
 array([[-4.01328046e+94, -1.78101108e+94],
        [-4.31998261e+94, -1.91711917e+94]]),
 array([[3.61891228e+94, 2.49462012e+94],
        [3.89547610e+94, 2.68526350e+94]]),
 array([[-3.28199253e+94, -2.46750903e+94],
        [-3.53280834e+94, -2.65608053e+94]])]

In [6]:
loss(coeffs)

1.474527577556881e+191